In [18]:
import csv
import requests
from bs4 import BeautifulSoup

### Função para acessar página, baixa código HTML e abrir um objeto BeautifulSoup

In [19]:
def prepara_sopa(url):
	# Cria objeto BeautifulSoup com o código HTML
	# url: string com o link da página
	# retorna objeto BeautifulSoup
	html = requests.get(url)
	sopa = BeautifulSoup(html.content, 'html.parser')
	return sopa

### Função para pegar lista de nomes e seus códigos

In [20]:
def dicio_menu(sopa, selector):
	# Função para pegar a lista do menu e seus valores
	# sopa: objeto BeautifulSoup
	# selector: string com seletor do menu
	# retorna: dicionário com par opção:valor
	lista_opcoes = sopa.select(selector)[0].find_all("option")
	dicio = {}
	for item in lista_opcoes:
		nome = item.text.encode('utf-8')
		valor = item.attrs['value'].encode('utf-8')
		dicio[nome] = valor
	return dicio

### Função para gerar URL usando código e data

In [21]:
def gera_url(conta, periodo):
	# Gera uma url usando o código da conta(gabinete) e o período selecionado
	# conta: string do seletor do menu "conta"
	# periodo: string do seletor do menu "periodo"
	# retorna string com URL
	url = 'http://transparencia.alesc.sc.gov.br/gabinetes.php?conta={0}&&periodo={1}'.format(conta, periodo)
	return url

### Função para extrair tabela da página e retornar dicionário com valores, incluindo gabinete e data

In [22]:
def extrai_tabela(sopa, conta, periodo):
	# Extrai tabela HTML da página e inclui conta e periodo
	# sopa: objeto BeautifulSoup
	# conta: string
	# periodo: string
	# retorna: lista - cada item é uma linha da tabela (a primeira é o cabeçalho)
	tabela_html = sopa.select('#conteudo table tr')
	tabela_final = []
	tabela_final.append(['Gabinete', 'Periodo', 'Grupo', 'Total'])

	for linha in tabela_html[1:]:
		valor_grupo = linha.select('td')[0].text.encode('utf-8')
		valor_total = linha.select('td')[1].text.strip().encode('utf-8')
		tabela_final.append([conta, periodo, valor_grupo, valor_total])

	return tabela_final

### Função para salvar lista no arquivo CSV

In [23]:
def salva_csv(tabela_final, header):
	# Salva uma lista de linhas no arquivo CSV
	# tabela_final: lista em que cada item é uma linha da tabela
	# header: booleano, indica se devemos escrever o cabeçalho ou não
	# retorna: nada
	with open('alesc_gabinetes.csv', 'a') as f:
		writer = csv.writer(f)
		if header is True:
			writer.writerow(tabela_final[0])
		writer.writerows(tabela_final[1:])
	return

### Acessar página e iniciar objeto BeautifulSoup

In [24]:
url = 'http://transparencia.alesc.sc.gov.br/gabinetes.php'
sopa = prepara_sopa(url)

### Guardar lista de nomes de deputados e seus códigos

In [25]:
seletor_conta = "#conta"
gabinetes = dicio_menu(sopa, seletor_conta)

### Guardar lista de datas e seus códigos

In [26]:
seletor_periodo = "#periodo"
datas = dicio_menu(sopa, seletor_periodo)
header = True

### Iterar sobre lista de deputados

In [30]:
for gabinete in gabinetes.items():
	# Iterar sobre listas de datas
	for data in datas.items():
		# Gerar URL certa usando deputado + data
		url = gera_url(gabinete[1],data[1])
		# Acessar site usando função e pegando o objeto BeautifulSoup
		sopa = prepara_sopa(url)
		# Aplicar função para extrair tabela e gerar dicionário
		tabela_final = extrai_tabela(sopa, gabinete[0], data[0])
		# Função para salvar o dicionário no arquivo CSV
		salva_csv(tabela_final, header)
		header = False
	# Exibe mensagem informando que terminou de gravar informações do gabinete
	print ("Finalizado: ", gabinete[0])

print ("Cabei! \o/")

Finalizado:  b'Selecione'
Finalizado:  b'Ada Lili Faraco Luca'
Finalizado:  b'Adilor Guglielmi'
Finalizado:  b'Aldo Schneider'
Finalizado:  b'Altair Guidi'
Finalizado:  b'Altair Silva'
Finalizado:  b'Amauri Soares'
Finalizado:  b'Ana Paula de Lima'
Finalizado:  b'Angela Albino'
Finalizado:  b'Ant\xc3\xb4nio Aguiar'
Finalizado:  b'Antonio Plinio de Castro Silva'
Finalizado:  b'Antonio Serafim venzon'
Finalizado:  b'Carlos Chiodini'
Finalizado:  b'Carlos Fernando Coruja Agustini'
Finalizado:  b'Cesar Souza Junior'
Finalizado:  b'Cesar Valduga'
Finalizado:  b'Ciro Roza'
Finalizado:  b'Cleiton Salvaro'
Finalizado:  b'Dalmo Claro de Oliveira'
Finalizado:  b'Daniel Tozzo'
Finalizado:  b'Darci de Matos'
Finalizado:  b'Dieter Janssen'
Finalizado:  b'Dirce Aparecida Heiderscheidt'
Finalizado:  b'Dirceu Luiz Dresch'
Finalizado:  b'Edison Adri\xc3\xa3o Andrino de Oliveira'
Finalizado:  b'Elizeu Mattos'
Finalizado:  b'Fabio Francisco Flor'
Finalizado:  b'Gabriel Ribeiro'
Finalizado:  b'Gean Lourei